# Unit 4 – Neural Network Training and Optimization

Arthur John Pagayon
BSCS 3-A AI

## Overview

In this exercise, I build a complete neural network from scratch and train it on the Spiral dataset. I explore different optimization techniques and compare their effectiveness.

**IMPORTANT: Install the following Python packages FIRST before running:**
1. **Numpy** - For numerical computations
2. **NNFS** - For the Spiral dataset
3. **scikit-learn** - (Optional) For additional datasets

## Key Learning Objectives
- Understand how backpropagation works
- Implement gradient descent with momentum
- Explore adaptive learning rates (Adagrad)
- Compare optimizer performance

In [ ]:
# Library imports
import numpy as np

## Step 1: Define Core Classes for Modularity

I'm implementing the fundamental building blocks of a neural network. Each class represents a specific layer type or activation function, making the code organized and reusable.

In [ ]:
# Generate spiral data for multi-class classification
def spiral_data(samples, classes):
    """
    Creates a spiral dataset with specified number of samples and classes.
    This is a classic non-linearly separable dataset used to test neural networks.
    """
    X = np.zeros((samples * classes, 2))
    y = np.zeros(samples * classes, dtype='uint8')
    for class_number in range(classes):
        ix = range(samples * class_number, samples * (class_number + 1))
        r = np.linspace(0.0, 1, samples)  # radius increases
        # Create spiral pattern with noise
        t = np.linspace(class_number * 4, (class_number + 1) * 4, samples) + np.random.randn(samples) * 0.2
        X[ix] = np.c_[r * np.sin(t * 2.5), r * np.cos(t * 2.5)]
        y[ix] = class_number
    return X, y


# Dense/Fully-connected layer
class Layer_Dense:
    """
    A fully connected layer performs: output = input @ weights + biases
    It stores inputs for the backward pass (backpropagation)
    """
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights with small random values
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        # Initialize biases to zero
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        """Compute output: Z = X·W + b"""
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        """
        Backward pass: compute gradients
        - dweights: gradient w.r.t. weights
        - dbiases: gradient w.r.t. biases  
        - dinputs: gradient w.r.t. inputs (for previous layer)
        """
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)


# ReLU activation: max(0, x) - introduces non-linearity
class Activation_ReLU:
    """
    ReLU (Rectified Linear Unit) activation function.
    Outputs x if x > 0, else 0. Helps networks learn non-linear patterns.
    """
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        """Gradient flows only for positive inputs"""
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


# Softmax activation: converts to probability distribution
class Activation_Softmax:
    """
    Softmax activation for multi-class classification.
    Converts raw outputs to probabilities that sum to 1.
    Formula: softmax(z) = exp(z) / sum(exp(z))
    """
    def forward(self, inputs):
        self.inputs = inputs
        # Subtract max for numerical stability
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        """Compute Jacobian matrix for each sample"""
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            # Jacobian = diag(p) - p·p^T
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


# Categorical Cross-Entropy loss for multi-class classification
class Loss_CategoricalCrossEntropy:
    """
    Loss function: -sum(y_true * log(y_pred))
    Measures how far the predicted probabilities are from the true labels.
    """
    def forward(self, y_pred, y_true):
        samples = y_pred.shape[0]
        # Clip predictions to avoid log(0)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        if len(y_true.shape) == 1:
            # Sparse labels (class indices)
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            # One-hot encoded labels
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        """Backward pass for loss"""
        samples = len(dvalues)
        labels = len(dvalues[0])
        
        # Convert to one-hot if sparse
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        # Gradient: -y_true / y_pred
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [ ]:
# Advanced Optimizer with multiple techniques
class Optimizer:
    """
    SGD Optimizer with support for:
    - Learning rate decay: reduces lr over time
    - Momentum: accelerates gradient in consistent directions
    - Adagrad: adaptive learning rate per parameter
    
    These techniques help optimization converge faster and reach better solutions.
    """
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0, use_adagrad=False, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay  # Learning rate decay rate
        self.iterations = 0
        self.momentum = momentum  # Momentum factor (0 = no momentum)
        self.use_adagrad = use_adagrad  # Adaptive gradient scaling
        self.epsilon = epsilon  # Small constant for numerical stability

    def pre_update_params(self):
        """Apply learning rate decay before each update"""
        if self.decay:
            self.current_learning_rate = self.learning_rate / (1.0 + self.decay * self.iterations)

    def update_params(self, layer):
        """Update layer parameters using chosen optimization strategy"""
        # Initialize momentum if needed
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

        # Initialize Adagrad cache if needed
        if self.use_adagrad:
            if not hasattr(layer, 'weight_cache'):
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)

        # Apply momentum (if enabled)
        if self.momentum:
            # Momentum update: v = β·v - lr·dW
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
            
            if self.use_adagrad:
                # Also apply Adagrad scaling
                layer.weight_cache += layer.dweights ** 2
                layer.bias_cache += layer.dbiases ** 2
                layer.weights += weight_updates / (np.sqrt(layer.weight_cache) + self.epsilon)
                layer.biases += bias_updates / (np.sqrt(layer.bias_cache) + self.epsilon)
            else:
                layer.weights += weight_updates
                layer.biases += bias_updates
        else:
            # No momentum - vanilla SGD or Adagrad
            if self.use_adagrad:
                layer.weight_cache += layer.dweights ** 2
                layer.bias_cache += layer.dbiases ** 2
                layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
                layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)
            else:
                layer.weights += -self.current_learning_rate * layer.dweights
                layer.biases += -self.current_learning_rate * layer.dbiases

    def post_update_params(self):
        """Increment iteration counter after updates"""
        self.iterations += 1

## Step 2: Training Loop and Evaluation

I now implement the complete training function that performs forward and backward passes. The network learns by adjusting weights and biases based on the computed gradients.

In [ ]:
def train(optimizer, epochs=1000, print_every=100):
    """
    Train the neural network on spiral data.
    
    Args:
        optimizer: Optimizer instance (SGD with various settings)
        epochs: Number of training iterations
        print_every: How often to print progress
    
    Returns:
        losses: List of loss values per epoch
        accuracy: Final accuracy achieved
        stable_epoch: Epoch where loss stabilized
    """
    # Load dataset
    X, y = spiral_data(samples=100, classes=3)

    # Build network: 2 inputs → 64 hidden → 3 outputs
    dense1 = Layer_Dense(2, 64)
    activation1 = Activation_ReLU()
    dense2 = Layer_Dense(64, 3)
    activation2 = Activation_Softmax()
    loss_function = Loss_CategoricalCrossEntropy()

    losses = []
    stable_epoch = None
    stable_counter = 0
    
    # Training loop
    for epoch in range(1, epochs + 1):
        # Step 1: Adjust learning rate based on decay
        optimizer.pre_update_params()

        # Step 2: Forward pass
        dense1.forward(X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        activation2.forward(dense2.output)

        # Step 3: Compute loss and accuracy
        loss = np.mean(loss_function.forward(activation2.output, y))
        losses.append(loss)

        predictions = np.argmax(activation2.output, axis=1)
        accuracy = np.mean(predictions == y)

        # Print progress
        if epoch % print_every == 0 or epoch == 1:
            print(f"Epoch {epoch}/{epochs} - loss: {loss:.4f} - acc: {accuracy:.4f} - lr: {optimizer.current_learning_rate:.6f}")

        # Step 4: Backward pass (compute gradients)
        loss_function.backward(activation2.output, y)
        dvalues = loss_function.dinputs
        activation2.backward(dvalues)
        dense2.backward(activation2.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)

        # Step 5: Update parameters using optimizer
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
        optimizer.post_update_params()

        # Step 6: Detect when training stabilizes
        if epoch > 1:
            if abs(losses[-1] - losses[-2]) < 1e-5:
                stable_counter += 1
            else:
                stable_counter = 0
            if stable_counter >= 10 and stable_epoch is None:
                stable_epoch = epoch - 10 + 1

    return losses, accuracy, stable_epoch

In [ ]:
## Step 3: Experiment with Different Optimizers

I'll train the network using two different optimization strategies and compare their performance.

# Set random seed for reproducibility
np.random.seed(0)

print("="*70)
print("NEURAL NETWORK TRAINING WITH DIFFERENT OPTIMIZERS")
print("="*70)

# Experiment 1: Vanilla SGD with learning rate decay
print('\n[1] Training with vanilla SGD (lr=1.0, decay=1e-3)')
print('-' * 70)
opt1 = Optimizer(learning_rate=1.0, decay=1e-3, momentum=0.0, use_adagrad=False)
losses1, acc1, stable1 = train(opt1, epochs=1000, print_every=100)

# Experiment 2: SGD with Momentum + Adagrad
print('\n\n[2] Training with momentum + Adagrad (lr=0.5, decay=1e-4, momentum=0.9)')
print('-' * 70)
opt2 = Optimizer(learning_rate=0.5, decay=1e-4, momentum=0.9, use_adagrad=True)
losses2, acc2, stable2 = train(opt2, epochs=1000, print_every=100)

# Summary and Analysis
print('\n' + "="*70)
print("TRAINING SUMMARY & ANALYSIS")
print("="*70)
print(f'\nVanilla SGD:')
print(f'  • Final Accuracy: {acc1:.4f} ({acc1*100:.2f}%)')
print(f'  • Stabilized at Epoch: {stable1}')

print(f'\nMomentum + Adagrad:')
print(f'  • Final Accuracy: {acc2:.4f} ({acc2*100:.2f}%)')
print(f'  • Stabilized at Epoch: {stable2}')

# Compare optimizers
improvement = (acc2 - acc1) * 100
if stable2 and stable1:
    speedup = stable1 - stable2
    print(f'\nComparison:')
    print(f'  • Accuracy Improvement: {improvement:.2f}%')
    print(f'  • Convergence Speedup: {speedup} epochs faster')
print("="*70)